In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///MOSFET Properties - Sample.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM 'MAIN TABLE' LIMIT 10;")

sqlite
['Main Table']


"[('1', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Continuous drain current 1)', 'ID', 'Min', '-', 'A', 'VGS=18 V, TC=25 °C VGS=18 V, TC=100 °C VGS=18 V, TA=25 °C, RTHJA=40 °C/W 2)'), ('2', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Continuous drain current 1)', 'ID', 'Typ.', '-', 'A', 'VGS=18 V, TC=25 °C VGS=18 V, TC=100 °C VGS=18 V, TA=25 °C, RTHJA=40 °C/W 2)'), ('3', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Continuous drain current 1)', 'ID', 'Max', '144 102 13.4', 'A', 'VGS=18 V, TC=25 °C VGS=18 V, TC=100 °C VGS=18 V, TA=25 °C, RTHJA=40 °C/W 2)'), ('4', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Pulsed drain current 3)', 'ID,pulse', 'Min', '-', 'A', 'TC=25 °C'), ('5', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Pulsed drain current 3)', 'ID,pulse', 'Typ.', '-', 'A', 'TC=25 °C'), ('6', 'IMT40R011M2H', '3', 'Table 2      Maximum ra tings', 'Pulsed drain current 3)', 'ID,pulse', 'Max', '432', 'A', 'TC=25 °C'), ('7', 'IMT40R0

In [2]:
from typing_extensions import TypedDict 
class State(TypedDict):
    question: str 
    query: str
    result: str 
    answer: str 

In [3]:
import getpass 
import os 

if not os.environ.get("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API Key for Groq: ")

from langchain.chat_models import init_chat_model 
llm = init_chat_model("llama3-8b-8192", model_provider="groq")

In [5]:
from langchain import hub 

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")
assert len(query_prompt_template.messages) == 1 
query_prompt_template.messages[0].pretty_print()

c:\Users\Ard\anaconda3\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [6]:
from typing_extensions import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": "MainTable",
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}

In [63]:
write_query({"question": "Given column names ItemID, Parameter, ParameterType, ParameterValue, Unit. What is the continuous drain current of IMT40R011M2H?"})

{'query': "SELECT Unit FROM MainTable WHERE ItemID = 'IMT40R011M2H' AND Parameter = 'continuous drain current'"}

In [7]:
system_prompt_additional = """Table columns are ItemID, Parameter, ParameterType, and ParameterValue. 
ItemID contains the ID of a Product, which is a long alphanumeric string.  
Parameter contains the name of a parameter. Parameter should be fuzzily searched only. 
ParameterType contains the type of the parameter, which is either Min, Max, or Typ. 
When writing SQL queries, always include the ItemID, Parameter, and ParameterType columns.
Generate an SQL query based on the question: What is the maximum continuous drain current of item IMT40R011M2H? """
query = write_query({"question": system_prompt_additional})
print(query['query'])


SELECT ItemID, Parameter, ParameterType FROM MainTable WHERE Parameter LIKE '%drain%' AND ParameterType='Max' AND ItemID LIKE '%IMT40R011M2H%' LIMIT 10


OperationalError: (sqlite3.OperationalError) near "FROM": syntax error
[SQL: SELECT  
FROM "Main Table"
 LIMIT ? OFFSET ?]
[parameters: (3, 0)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)